# 영화 추천: ranking 단계

실제 추천 시스템은 종종 두 단계로 구성됩니다.

1. 검색 단계(Retrieval Phase)에서는 가능한 모든 후보 중에서 수백개의 초기 후보 집합을 선택합니다. 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보를 효율적으로 제거하는 것입니다. 검색 모델은 수백만 개의 후보를 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

2. 순위 단계(Ranking Phase)에서는 검색 모델의 출력을 가져와서 가능한 한 가장 좋은 추천 항목을 선택하도록 미세 조정합니다. 이 작업은 사용자가 관심을 가질 만한 항목 집합을 가능한 후보의 최종 목록으로 좁히는 것입니다.

이 notebook 은 두번째 단계인 순위 단계 모델을 구현합니다.  

다음을 수행합니다.

1. 데이터를 가져와 훈련 세트와 테스트 세트로 나눕니다.
2. 순위 모델을 구현합니다.
3. 피팅하고 평가합니다.


In [ ]:
# TFRS와 호환되는 이전 version 설치
# 꼭 다시 시작한 직후 실행하세요!
!pip install --upgrade --force-reinstall -q \
  tensorflow==2.15.0 \
  tensorflow-recommenders==0.7.3 \
  tensorflow-datasets==4.9.4 \
  protobuf==3.20.* \
  ml-dtypes==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [ ]:
import os
# 데이터를 보기 좋게 출력하기 위한 모듈
import pprint
# 임시 파일 및 디렉토리 생성을 위한 모듈
import tempfile
# 타입 힌트를 위한 모듈
from typing import Dict, Text

import numpy as np
import tensorflow as tf
# TensorFlow 데이터셋을 사용하기 위한 라이브러리
import tensorflow_datasets as tfds
# 추천 시스템을 구축하기 위한 TensorFlow 확장 라이브러리
import tensorflow_recommenders as tfrs

# Version 확인
print("TF:", tf.__version__)
print("TFRS:", tfrs.__version__)
print("TFDS:", tfds.__version__)

TF: 2.15.0
TFRS: v0.7.3
TFDS: 4.9.4


## Preparing the dataset

[retrieval](basic_retrieval) 과 동일한 데이터를 사용합니다. 이번에는 등급(explicit feedback)도 유지합니다. 이는 우리가 예측하려는 target 입니다.

In [ ]:
# MovieLens 100K 데이터셋에서 'ratings' 정보를 로드합니다. 데이터는 훈련용 분할로 지정됩니다.
ratings = tfds.load("movielens/100k-ratings", split="train")

# 로드된 'ratings' 데이터에서 필요한 특성만 추출하여 새로운 딕셔너리 구조로 매핑합니다.
# 'map' 함수를 사용하여 각 데이터 요소를 원하는 형태로 변환합니다.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],  # 영화 제목
    "user_id": x["user_id"],          # 사용자 ID
    "user_rating": x["user_rating"]   # 사용자가 영화에 부여한 평점
})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/0.1.1.incompleteZP8L0Y/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


이전과 마찬가지로 train/test를 80% 대 20%로 분할합니다.

In [ ]:
# 랜덤 시드를 설정
tf.random.set_seed(42)

# ratings 데이터셋을 100,000개의 버퍼 크기로 무작위로 섞습니다.
# reshuffle_each_iteration=False는 매 반복(iteration)마다 데이터셋을 다시 섞지 않도록 설정합니다.
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# 섞인 데이터셋에서 첫 80,000개를 훈련 데이터로 가져옵니다.
train = shuffled.take(80_000)

# 섞인 데이터셋에서 첫 80,000개 이후의 데이터 중 20,000개를 테스트 데이터로 가져옵니다.
test = shuffled.skip(80_000).take(20_000)

# 훈련 데이터셋과 테스트 데이터셋의 길이를 계산합니다.
# TensorFlow 데이터셋에는 len() 함수를 직접 사용할 수 없으므로, cardinality() 메서드를 사용하여 각 데이터셋의 크기를 확인합니다.
train_length = train.cardinality().numpy()
test_length = test.cardinality().numpy()

print("train dataset 길이:", train_length)
print("test dataset 길이:", test_length)

train dataset 길이: 80000
test dataset 길이: 20000


In [ ]:
# ratings 데이터셋에서 영화 제목을 추출합니다. 대량의 데이터를 처리하기 위해 큰 배치 크기를 설정합니다.
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])

# ratings 데이터셋에서 사용자 ID를 추출합니다. 마찬가지로 큰 배치 크기를 설정하여 처리합니다.
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

# 영화 제목과 사용자 ID의 중복을 제거합니다.
# TensorFlow 데이터셋을 list로 변환한 후 NumPy의 unique 함수를 사용하여 중복을 제거합니다.
unique_movie_titles = np.unique(np.array(list(movie_titles.as_numpy_iterator())))
unique_user_ids = np.unique(np.array(list(user_ids.as_numpy_iterator())))

# 중복 제거 후의 영화 제목 중 처음 10개를 출력합니다.
print("Unique movie titles:", unique_movie_titles[:10])

Unique movie titles: [b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)' b'2 Days in the Valley (1996)'
 b'20,000 Leagues Under the Sea (1954)' b'2001: A Space Odyssey (1968)'
 b'3 Ninjas: High Noon At Mega Mountain (1998)' b'39 Steps, The (1935)']


## 모델 구현

### Architecture

랭킹 모델은 검색 모델처럼 효율성 제약에 직면하지 않으므로 아키텍처를 선택할 때 조금 더 자유로워집니다.

여러 스택의 dense layer로 구성된 모델은 task 순위 지정을 위한 비교적 일반적인 아키텍처입니다. 다음과 같이 구현할 수 있습니다.

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    # 임베딩 차원을 32로 설정
    embedding_dimension = 32

    # 사용자에 대한 임베딩
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        # 사용자 ID를 정수 인덱스로 변환
        vocabulary=unique_user_ids, mask_token=None),
        # 사용자 임베딩 레이어
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # 영화에 대한 임베딩
    self.movie_embeddings = tf.keras.Sequential([
      #  영화 제목을 정수 인덱스로 변환
      tf.keras.layers.StringLookup(
         vocabulary=unique_movie_titles, mask_token=None),
      # 영화 임베딩 레이어
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # 평점 예측 신경망
    self.ratings = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(1)
      ])

  def call(self, inputs):

    user_id, movie_title = inputs

    # 입력받은 사용자 ID와 영화 제목에 대한 임베딩을 가져옵니다.
    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    # 사용자 임베딩과 영화 임베딩을 연결(concatenate)하고 평점을 예측
    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

이 모델은 user ID와 movie title을 가져와 예상 등급을 출력합니다.

In [ ]:
# RankingModel 인스턴스를 생성하고, 사용자 ID와 영화 제목을 입력으로 전달하여 호출
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01461909]], dtype=float32)>

### Loss and metrics

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 작업이 있습니다.

이 경우 손실 함수와 메트릭 계산을 함께 묶는 편리한 래퍼인 `Ranking` 작업 개체를 사용합니다.

평점을 예측하기 위해 `MeanSquaredError` Keras 손실과 함께 사용할 것입니다.

In [ ]:
# 순위 매기기 작업을 정의
task = tfrs.tasks.Ranking(
  # 손실 함수로 평균 제곱 오차(Mean Squared Error)를 사용합니다.
  # 이는 예측 평점과 실제 평점 간의 차이의 제곱에 대한 평균을 계산합니다.
  loss = tf.keras.losses.MeanSquaredError(),

  # 평가 지표로는 루트 평균 제곱 오차(Root Mean Squared Error)를 사용합니다.
  # RMSE는 MSE의 제곱근으로, 오차의 크기를 원래의 단위로 복원하여 해석하기 쉽게 만듭니다.
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

task 자체는 true 와 predicted 를 인수로 받아 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 빌딩 모델을 간소화하는 base 모델 클래스(`tfrs.models.Model`)를 노출합니다. feature를 제공하고 손실 값을 반환합니다.

그러면 base model이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

In [ ]:
# 사용자가 영화에 부여할 평점을 예측하는 Full Model
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    # RankingModel 클래스를 사용하여 순위 매기기 모델을 정의
    self.ranking_model: tf.keras.Model = RankingModel()
    # 순위 매기기 작업을 정의합니다. 손실 함수와 평가 지표를 지정합니다.
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
       # 손실 함수로 평균 제곱 오차를 사용
       loss = tf.keras.losses.MeanSquaredError(),
       # 평가 지표로 RMSE를 사용
       metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # 모델에 입력으로 사용자 ID와 영화 제목을 전달하여 평점 예측을 수행
    return self.ranking_model((features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 실제 사용자 평점을 가져옵니다.
    labels = features.pop("user_rating")

    # 모델을 호출하여 평점 예측을 수행
    rating_predictions = self(features)

    # 순위 매기기 작업(task)을 사용하여 손실을 계산
    # 이는 예측 평점과 실제 평점 간의 손실 및 평가 지표를 계산
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

In [ ]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

In [ ]:
# 훈련 데이터셋을 섞고 배치 처리한 후 캐시합니다.
# 배치 처리는 모델 학습이나 평가 시 여러 샘플을 동시에 처리하도록 하며, 캐싱은 반복 평가 시 성능을 향상시킵니다.
cached_train = train.shuffle(100_000).batch(8192).cache()

cached_test = test.batch(4096).cache()

 모델을 훈련합니다.

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 4s 41ms/step - root_mean_squared_error: 2.0723 - loss: 3.9741 - regularization_loss: 0.0000e+00 - total_loss: 3.9741
Epoch 2/3
10/10 [==============================] - 0s 27ms/step - root_mean_squared_error: 1.1776 - loss: 1.3773 - regularization_loss: 0.0000e+00 - total_loss: 1.3773
Epoch 3/3
10/10 [==============================] - 0s 38ms/step - root_mean_squared_error: 1.1171 - loss: 1.2482 - regularization_loss: 0.0000e+00 - total_loss: 1.2482


In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 13ms/step - root_mean_squared_error: 1.1047 - loss: 1.2156 - regularization_loss: 0.0000e+00 - total_loss: 1.2156


{'root_mean_squared_error': 1.1047029495239258,
 'loss': 1.1948188543319702,
 'regularization_loss': 0,
 'total_loss': 1.1948188543319702}

RMSE 측정항목이 낮을수록 모델이 등급을 예측하는 경우가 더 정확해집니다.

## Testing the ranking model

이제 영화 세트에 대한 예측을 계산하여 ranking model을 테스트한 다음 예측을 기반으로 이러한 영화의 순위를 지정할 수 있습니다.


In [ ]:
test_ratings = {}

# 추천 시스템에서 검색 모델을 통해 얻은 추천 영화 후보 목록입니다.
test_movie_titles = [
    'Bedknobs and Broomsticks (1971)', 'Aristocats, The (1970)', 'Sound of Music, The (1965)',
    "Kid in King Arthur's Court, A (1995)", 'Angels in the Outfield (1994)', 'Cool Runnings (1993)',
    'Princess Caraboo (1994)', 'Rent-a-Kid (1995)', 'Fried Green Tomatoes (1991)', 'Nell (1994)'
]

# 각 영화 제목에 대해 평가 모델을 사용하여 사용자 42의 평점을 예측합니다.
for movie_title in test_movie_titles:
    # 평가 모델에 사용자 ID와 영화 제목을 입력하여 평점을 예측합니다.
    test_ratings[movie_title] = model({
        "user_id": np.array(["42"]),
        "movie_title": np.array([movie_title])
    })

# 예측된 평점을 출력합니다. 출력은 평점이 높은 순으로 정렬됩니다.
print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
    print(f"{title}: {score}")

Ratings:
Rent-a-Kid (1995): [[3.6042027]]
Sound of Music, The (1965): [[3.543466]]
Cool Runnings (1993): [[3.5380356]]
Aristocats, The (1970): [[3.5260172]]
Bedknobs and Broomsticks (1971): [[3.5182977]]
Fried Green Tomatoes (1991): [[3.4985693]]
Kid in King Arthur's Court, A (1995): [[3.4652867]]
Princess Caraboo (1994): [[3.4580193]]
Angels in the Outfield (1994): [[3.4388006]]
Nell (1994): [[3.4318926]]
